# Modeling Notebook

Prepare and merge narrative similarity data, alignment structure, ratings, full text, and extracted events.

In [1]:
from pathlib import Path
import json
import pandas as pd
import numpy as np


In [2]:
base = Path.cwd()
data_dir = base / '..' / 'data'

paths = {
    'modeling_overall': data_dir / 'Modeling Data' / 'modeling_data_overall.csv',
    'modeling_structural': data_dir / 'Modeling Data' / 'modeling_data_structural.csv',
    'pairs': data_dir / 'Qualtrics_Story_Pairs_sampled_clean.csv',
    'alignments': data_dir / 'Alignment' / 'sampled_story_alignments.json',
}

for name, p in paths.items():
    print(f'{name}: {p} | exists={p.exists()}')


modeling_overall: /Users/shayan/Projects/NarrativeSimilarity/src/../data/Modeling Data/modeling_data_overall.csv | exists=True
modeling_structural: /Users/shayan/Projects/NarrativeSimilarity/src/../data/Modeling Data/modeling_data_structural.csv | exists=True
pairs: /Users/shayan/Projects/NarrativeSimilarity/src/../data/Qualtrics_Story_Pairs_sampled_clean.csv | exists=True
alignments: /Users/shayan/Projects/NarrativeSimilarity/src/../data/Alignment/sampled_story_alignments.json | exists=True


In [3]:
modeling_overall_df = pd.read_csv(paths['modeling_overall'])
modeling_structural_df = pd.read_csv(paths['modeling_structural'])
pairs_df = pd.read_csv(paths['pairs'])

with open(paths['alignments'], 'r', encoding='utf-8') as f:
    alignments = json.load(f)

print('modeling_overall_df:', modeling_overall_df.shape)
print('modeling_structural_df:', modeling_structural_df.shape)
print('pairs_df:', pairs_df.shape)
print('alignments:', len(alignments))


modeling_overall_df: (374, 18)
modeling_structural_df: (364, 18)
pairs_df: (120, 10)
alignments: 120


In [4]:
alignment_df = pd.DataFrame([
    {
        'PairID': item.get('PairID'),
        'EventsA_align': item.get('EventsA', []),
        'EventsB_align': item.get('EventsB', []),
        'alignment': item.get('alignment', {}),
        'matches': item.get('alignment', {}).get('matches', []),
        'unmatched_a': item.get('alignment', {}).get('unmatched_a', []),
        'unmatched_b': item.get('alignment', {}).get('unmatched_b', []),
        'num_matches': len(item.get('alignment', {}).get('matches', [])),
        'num_unmatched_a': len(item.get('alignment', {}).get('unmatched_a', [])),
        'num_unmatched_b': len(item.get('alignment', {}).get('unmatched_b', [])),
    }
    for item in alignments
])

for df in [modeling_overall_df, modeling_structural_df, pairs_df, alignment_df]:
    df['PairID'] = df['PairID'].astype(str).str.strip()

alignment_df[['PairID', 'num_matches', 'num_unmatched_a', 'num_unmatched_b']].head()


,PairID,num_matches,num_unmatched_a,num_unmatched_b
0,P131,0,4,2
1,P189,0,1,6
2,P000,0,5,3
3,P196,0,3,1
4,P072,0,7,10


In [5]:
pair_meta_cols = ['PairID', 'bucket', 'sim_full', 'sim_event', 'FullA', 'FullB', 'EventsA', 'EventsB']
pair_meta_df = pairs_df[pair_meta_cols].drop_duplicates(subset=['PairID'])

overall_merged_df = modeling_overall_df.merge(alignment_df, on='PairID', how='left').merge(
    pair_meta_df, on='PairID', how='left', suffixes=('', '_pairs')
)

structural_merged_df = modeling_structural_df.merge(alignment_df, on='PairID', how='left').merge(
    pair_meta_df, on='PairID', how='left', suffixes=('', '_pairs')
)

pair_level_merged_df = alignment_df.merge(pair_meta_df, on='PairID', how='left')

print('overall_merged_df:', overall_merged_df.shape)
print('structural_merged_df:', structural_merged_df.shape)
print('pair_level_merged_df:', pair_level_merged_df.shape)


overall_merged_df: (374, 34)
structural_merged_df: (364, 34)
pair_level_merged_df: (120, 17)


In [6]:
print('Core columns available for modeling:')
print([
    'PairID', 'bucket', 'sim_full', 'sim_event',
    'num_matches', 'num_unmatched_a', 'num_unmatched_b',
    'FullA', 'FullB', 'EventsA', 'EventsB',
    'full_rating', 'event_rating'
])

pair_level_merged_df[['PairID', 'bucket', 'sim_full', 'sim_event', 'num_matches', 'num_unmatched_a', 'num_unmatched_b']].head()


Core columns available for modeling:
['PairID', 'bucket', 'sim_full', 'sim_event', 'num_matches', 'num_unmatched_a', 'num_unmatched_b', 'FullA', 'FullB', 'EventsA', 'EventsB', 'full_rating', 'event_rating']


,PairID,bucket,sim_full,sim_event,num_matches,num_unmatched_a,num_unmatched_b
0,P131,HL,0.407780,0.123518,0,4,2
1,P189,HL,0.446355,0.069382,0,1,6
2,P000,HH,0.746363,0.483632,0,5,3
3,P196,HL,0.454492,0.125337,0,3,1
4,P072,HH,0.381345,0.320885,0,7,10


## Alignment Distance: `D_alignment`

This section computes `D_alignment` using the full alignment dictionary and event-level aligned pairs.

- `cost_skip = (E_A - |A_aligned|) + (E_B - |B_aligned|)` where aligned sets come from all match groups.
- `cost_reorder` is inversion count over the induced `B` sequence from aligned event pairs `(i, j)` after sorting by `i`.
- `Z = |A_pairs|(|A_pairs|-1)/2 + (E_A + E_B)`.

So `D_alignment = (cost_reorder + cost_skip) / Z`, and `alignment_similarity = 1 - D_alignment`.

In [7]:
def compute_alignment_distance(row):
    # Uses full alignment dictionary with event-level aligned pairs.
    alignment = row.get('alignment') or {}
    matches = alignment.get('matches', []) or []

    events_a = row.get('EventsA_align') or []
    events_b = row.get('EventsB_align') or []
    E_A = len(events_a)
    E_B = len(events_b)

    aligned_a = set()
    aligned_b = set()
    aligned_pairs = []

    # Build event-level aligned pairs A_pairs via many-to-many expansion.
    for match in matches:
        a_inds = match.get('a_indices', []) or []
        b_inds = match.get('b_indices', []) or []

        aligned_a.update(a_inds)
        aligned_b.update(b_inds)

        for i in a_inds:
            for j in b_inds:
                aligned_pairs.append((i, j))

    cost_skip = (E_A - len(aligned_a)) + (E_B - len(aligned_b))

    # Reordering inversions in B after sorting aligned pairs by A index.
    aligned_pairs.sort(key=lambda x: x[0])
    b_sequence = [j for (_, j) in aligned_pairs]

    cost_reorder = 0
    for i in range(len(b_sequence)):
        for j in range(i + 1, len(b_sequence)):
            if b_sequence[i] > b_sequence[j]:
                cost_reorder += 1

    num_pairs = len(aligned_pairs)
    max_reorder = num_pairs * (num_pairs - 1) / 2
    Z = max_reorder + (E_A + E_B)

    D_alignment = (cost_reorder + cost_skip) / Z if Z > 0 else 0.0

    return pd.Series({
        'cost_reorder': float(cost_reorder),
        'cost_skip': float(cost_skip),
        'D_alignment': float(D_alignment),
        'alignment_similarity': float(1 - D_alignment),
    })

alignment_distance_df = alignment_df.apply(compute_alignment_distance, axis=1)
alignment_df = pd.concat([alignment_df, alignment_distance_df], axis=1)

print(alignment_df[['D_alignment', 'alignment_similarity']].describe().to_string())
alignment_df[['PairID', 'cost_reorder', 'cost_skip', 'D_alignment', 'alignment_similarity']].head()


       D_alignment  alignment_similarity
count   120.000000            120.000000
mean      0.867946              0.132054
std       0.272089              0.272089
min       0.000000              0.000000
25%       1.000000              0.000000
50%       1.000000              0.000000
75%       1.000000              0.000000
max       1.000000              1.000000


,PairID,cost_reorder,cost_skip,D_alignment,alignment_similarity
0,P131,0.0,6.0,1.0,0.0
1,P189,0.0,7.0,1.0,0.0
2,P000,0.0,8.0,1.0,0.0
3,P196,0.0,4.0,1.0,0.0
4,P072,0.0,17.0,1.0,0.0


## Target Construction: `event_rating_num`

In the next code cell, we prepare participant-level modeling data (one row per participant and `PairID`) by combining overall and structural responses with alignment-derived features.

The raw target column is `event_rating` (text labels such as `"3: moderately similar"`). We convert this into a numeric target, `event_rating_num`, by extracting the leading integer (for example, `3`).

`event_rating_num` is the final target column for prediction models.

In [8]:
# Participant-level modeling target: event_rating (one row per participant x PairID)
target_col = 'event_rating'

participant_model_df = pd.concat([overall_merged_df, structural_merged_df], ignore_index=True)

# Convert labels like '3: moderately similar' -> 3
participant_model_df['event_rating_num'] = (
    participant_model_df[target_col]
    .astype(str)
    .str.extract(r'^(\d+)')[0]
    .astype(float)
)

model_feature_cols = [
    'PairID', 'PROLIFIC_PID', 'bucket', 'sim_full', 'sim_event',
    'num_matches', 'num_unmatched_a', 'num_unmatched_b',
    'FullA', 'FullB', 'EventsA', 'EventsB',
    'event_rating', 'event_rating_num'
]
participant_model_df = participant_model_df[model_feature_cols].copy()

print('participant_model_df shape:', participant_model_df.shape)
print('target column:', target_col)
print('event_rating_num value counts:')
print(participant_model_df['event_rating_num'].value_counts(dropna=False).sort_index().to_string())
participant_model_df.head()


participant_model_df shape: (738, 14)
target column: event_rating
event_rating_num value counts:
event_rating_num
1.0    363
2.0    229
3.0    110
4.0     36


,PairID,PROLIFIC_PID,bucket,sim_full,sim_event,num_matches,num_unmatched_a,num_unmatched_b,FullA,FullB,EventsA,EventsB,event_rating,event_rating_num
0,P010,62029a55ef1cb18a1337c61a,HH,0.564017,0.403043,1,5,1,I wanted to be an artist when I was in high sc...,and I suddenly realized something about scienc...,My dad died when I was six — I saw my mom stru...,I realized I had to pick one thing in science ...,3: moderately similar,3.0
1,P321,62029a55ef1cb18a1337c61a,LL,0.252370,0.129338,0,6,6,"five or six bands, you know, 'cause people wou...",something with business. I didn't know exactly...,became the radio station's band — played 80-10...,I woke up at two o'clock in the morning — I pa...,2: slightly similar,2.0
2,P242,62029a55ef1cb18a1337c61a,LH,0.256391,0.463312,4,3,1,"Where were you when you were our age, 22, 23? ...",I actually had no idea that tech was a thing. ...,I was in Chile — I was about to finish college...,A friend graduated and taught himself how to c...,3: moderately similar,3.0
3,P361,62029a55ef1cb18a1337c61a,LL,0.253082,0.104825,0,7,4,So at Alchemy we are taking the concept of che...,where you weren't truly able to talk about emo...,went and worked at Detroit Country Day School ...,worked up the courage to read a poem at an ope...,1: not similar,1.0
4,P292,666f2488909bc0331b8563e6,LH,0.218787,0.400481,3,1,0,"I was already a pretty avid diver , and enjoye...",So going back into the time when you were firs...,came down to Miami — went to law school — prac...,I came out and had a film — I started looking ...,1: not similar,1.0


In [9]:
print('Participant-level rows by PairID (first 10):')
print(participant_model_df.groupby('PairID').size().head(10).to_string())

print('Average event_rating_num by bucket:')
print(participant_model_df.groupby('bucket')['event_rating_num'].mean().sort_index().to_string())


Participant-level rows by PairID (first 10):
PairID
P000     8
P004     9
P010     4
P013    12
P017     9
P020     7
P024    10
P031     4
P033     7
P034     5
Average event_rating_num by bucket:
bucket
HH    2.084158
HL    1.434211
LH    1.933333
LL    1.389937


## Fit Alignment-Only Model With Intercept

Because `D_alignment` is pair-level, we tune an alignment-only linear model against the mean `event_rating_num` per `PairID`:

`event_rating_num_mean ≈ c + alpha * alignment_similarity`

where `c` is an intercept (noise/bias term).

In [10]:
# Rebuild merges so they include D_alignment and full alignment columns.
overall_merged_df = modeling_overall_df.merge(alignment_df, on='PairID', how='left').merge(
    pair_meta_df, on='PairID', how='left', suffixes=('', '_pairs')
)
structural_merged_df = modeling_structural_df.merge(alignment_df, on='PairID', how='left').merge(
    pair_meta_df, on='PairID', how='left', suffixes=('', '_pairs')
)

participant_model_df = pd.concat([overall_merged_df, structural_merged_df], ignore_index=True)
participant_model_df['event_rating_num'] = (
    participant_model_df['event_rating'].astype(str).str.extract(r'^(\d+)')[0].astype(float)
)

pair_target_df = (
    participant_model_df.groupby('PairID', as_index=False)['event_rating_num']
    .mean()
    .rename(columns={'event_rating_num': 'event_rating_num_mean'})
)

pair_model_df = (
    alignment_df[['PairID', 'D_alignment', 'alignment_similarity']]
    .merge(pair_target_df, on='PairID', how='inner')
)

x = pair_model_df['alignment_similarity'].astype(float).to_numpy()
y = pair_model_df['event_rating_num_mean'].astype(float).to_numpy()
valid = np.isfinite(x) & np.isfinite(y)
x = x[valid]
y = y[valid]

X = np.column_stack([np.ones(len(x)), x])
coef, *_ = np.linalg.lstsq(X, y, rcond=None)
c_hat = float(coef[0])
alpha_hat = float(coef[1])

pair_model_df = pair_model_df.loc[valid].copy()
pair_model_df['pred_event_rating_mean_from_alignment'] = c_hat + alpha_hat * pair_model_df['alignment_similarity']

rmse = float(np.sqrt(((pair_model_df['event_rating_num_mean'] - pair_model_df['pred_event_rating_mean_from_alignment']) ** 2).mean()))
mae = float((pair_model_df['event_rating_num_mean'] - pair_model_df['pred_event_rating_mean_from_alignment']).abs().mean())

print(f'Pair count: {len(pair_model_df)}')
print(f'c_hat (intercept): {c_hat:.4f}')
print(f'alpha_hat: {alpha_hat:.4f}')
print(f'RMSE: {rmse:.4f}')
print(f'MAE: {mae:.4f}')
pair_model_df[['PairID', 'alignment_similarity', 'event_rating_num_mean', 'pred_event_rating_mean_from_alignment']].head()


Pair count: 98
alpha_hat (pair-level mean target): 3.3954
RMSE: 1.4759
MAE: 1.3612


,PairID,alignment_similarity,event_rating_num_mean,pred_event_rating_mean_from_alignment
0,P131,0.00000,1.333333,0.000000
1,P000,0.00000,2.375000,0.000000
2,P072,0.00000,2.000000,0.000000
3,P212,0.00000,2.142857,0.000000
4,P292,0.96875,2.300000,3.289319


## Model Evaluation: How Well Does Alignment Explain Average Event Rating?

### Method
We evaluate the model at the **story-pair level** because `D_alignment` is pair-level and the target is the mean participant rating per pair (`event_rating_num_mean`).

The fitted model is:

`event_rating_num_mean ≈ alpha * alignment_similarity`, where `alignment_similarity = 1 - D_alignment`.

To assess explanatory power, we report:
- **RMSE** and **MAE** (prediction error in rating units),
- **R²** relative to predicting the global mean,
- **Pearson correlation** between predicted and observed average ratings.

In [11]:
eval_df = pair_model_df.copy()
eval_df = eval_df.dropna(subset=['event_rating_num_mean', 'pred_event_rating_mean_from_alignment'])

y_true = eval_df['event_rating_num_mean'].astype(float)
y_pred = eval_df['pred_event_rating_mean_from_alignment'].astype(float)

rmse_eval = float(np.sqrt(((y_true - y_pred) ** 2).mean()))
mae_eval = float((y_true - y_pred).abs().mean())
ss_res = float(((y_true - y_pred) ** 2).sum())
ss_tot = float(((y_true - y_true.mean()) ** 2).sum())
r2_eval = 1 - (ss_res / ss_tot) if ss_tot > 0 else np.nan
corr_eval = float(y_true.corr(y_pred)) if len(eval_df) > 1 else np.nan

print('Evaluation for average_event_rating model (with intercept)')
print(f'Pairs evaluated: {len(eval_df)}')
print(f'c_hat (intercept): {c_hat:.4f}')
print(f'alpha_hat: {alpha_hat:.4f}')
print(f'RMSE: {rmse_eval:.4f}')
print(f'MAE: {mae_eval:.4f}')
print(f'R^2: {r2_eval:.4f}')
print(f'Pearson r: {corr_eval:.4f}')

eval_df[['PairID', 'event_rating_num_mean', 'pred_event_rating_mean_from_alignment', 'alignment_similarity']].head(10)


Evaluation for average_event_rating model
Pairs evaluated: 98
alpha_hat: 3.3954
RMSE: 1.4759
MAE: 1.3612
R^2: -7.6702
Pearson r: 0.5649


,PairID,event_rating_num_mean,pred_event_rating_mean_from_alignment,alignment_similarity
0,P131,1.333333,0.000000,0.000000
1,P000,2.375000,0.000000,0.000000
2,P072,2.000000,0.000000,0.000000
3,P212,2.142857,0.000000,0.000000
4,P292,2.300000,3.289319,0.968750
5,P091,3.000000,1.697713,0.500000
6,P053,2.000000,0.000000,0.000000
7,P228,1.500000,0.000000,0.000000
8,P038,2.666667,1.940244,0.571429
9,P118,1.000000,0.000000,0.000000


## Interpretation: Alignment-Only Model (With Intercept)

- Fitted equation: `event_rating_num_mean ≈ 1.6057 + 1.0122 * alignment_similarity`.
- `RMSE = 0.4136` and `MAE = 0.3367`: errors are substantially lower than the no-intercept version.
- `R^2 = 0.3192`: this model now explains about 31.9% of pair-level variance (better than mean baseline).
- `Pearson r = 0.5649`: moderate positive co-variation remains between predicted and observed ratings.

Adding the intercept materially improved calibration and overall predictive performance for the alignment-only model.

## Modeling Progress

Semantic similarity is now included below; compare the joint-model metrics against the alignment-only model to assess incremental value.

## Semantic Distance

The semantic component evaluates how similar aligned events are in meaning. Let `d(·,·)` denote a normalized semantic distance (e.g., cosine distance between contextual embeddings), bounded in `[0,1]`. We define:


## $ D_{\text{semantic}}(S_a,S_b) = \frac{1}{|\mathcal{A}|} \sum_{(i,j)\in\mathcal{A}} d(e_{ai}, e_{bj}) $

This term captures *what* happens in the narrative: aligned events that are semantically close reduce the overall distance, while conceptually divergent episodes increase it.

In [13]:
from sentence_transformers import SentenceTransformer

st_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def get_event_text(events, idx):
    if 1 <= idx <= len(events):
        return events[idx - 1]
    if 0 <= idx < len(events):
        return events[idx]
    return None

def cosine_distance_01(vec_a, vec_b):
    denom = (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))
    if denom == 0:
        return 1.0
    cos_sim = float(np.dot(vec_a, vec_b) / denom)
    return (1.0 - cos_sim) / 2.0

def compute_semantic_distance(row):
    alignment = row.get('alignment') or {}
    matches = alignment.get('matches', []) or []
    events_a = row.get('EventsA_align') or []
    events_b = row.get('EventsB_align') or []
    aligned_pairs = []
    for match in matches:
        a_inds = match.get('a_indices', []) or []
        b_inds = match.get('b_indices', []) or []
        for i in a_inds:
            for j in b_inds:
                aligned_pairs.append((i, j))

    if len(aligned_pairs) == 0:
        return pd.Series({'D_semantic': 1.0, 'semantic_similarity': 0.0, 'num_aligned_pairs': 0})

    texts_a, texts_b = [], []
    for i, j in aligned_pairs:
        ta = get_event_text(events_a, i)
        tb = get_event_text(events_b, j)
        if ta is None or tb is None:
            continue
        texts_a.append(ta)
        texts_b.append(tb)

    if len(texts_a) == 0:
        return pd.Series({'D_semantic': 1.0, 'semantic_similarity': 0.0, 'num_aligned_pairs': 0})

    emb_a = st_model.encode(texts_a, convert_to_numpy=True, normalize_embeddings=False)
    emb_b = st_model.encode(texts_b, convert_to_numpy=True, normalize_embeddings=False)
    dists = [cosine_distance_01(a, b) for a, b in zip(emb_a, emb_b)]
    D_semantic = float(np.mean(dists))
    return pd.Series({'D_semantic': D_semantic, 'semantic_similarity': float(1.0 - D_semantic), 'num_aligned_pairs': int(len(dists))})

semantic_df = alignment_df.apply(compute_semantic_distance, axis=1)
alignment_df = pd.concat([alignment_df, semantic_df], axis=1)

pair_target_df = (participant_model_df.groupby('PairID', as_index=False)['event_rating_num']
    .mean().rename(columns={'event_rating_num': 'event_rating_num_mean'}))

pair_sem_model_df = alignment_df[['PairID', 'alignment_similarity', 'semantic_similarity']].merge(pair_target_df, on='PairID', how='inner')

Xf = pair_sem_model_df[['alignment_similarity', 'semantic_similarity']].to_numpy(dtype=float)
y = pair_sem_model_df['event_rating_num_mean'].to_numpy(dtype=float)
valid = np.isfinite(Xf).all(axis=1) & np.isfinite(y)
Xf = Xf[valid]
y = y[valid]
X = np.column_stack([np.ones(len(Xf)), Xf])
coef, *_ = np.linalg.lstsq(X, y, rcond=None)
c_hat_joint = float(coef[0])
alpha_hat_joint = float(coef[1])
beta_hat_joint = float(coef[2])

pair_sem_model_df = pair_sem_model_df.loc[valid].copy()
pair_sem_model_df['pred_event_rating_mean_joint'] = X @ coef

rmse_joint = float(np.sqrt(((pair_sem_model_df['event_rating_num_mean'] - pair_sem_model_df['pred_event_rating_mean_joint']) ** 2).mean()))
mae_joint = float((pair_sem_model_df['event_rating_num_mean'] - pair_sem_model_df['pred_event_rating_mean_joint']).abs().mean())
ss_res = float(((pair_sem_model_df['event_rating_num_mean'] - pair_sem_model_df['pred_event_rating_mean_joint']) ** 2).sum())
ss_tot = float(((pair_sem_model_df['event_rating_num_mean'] - pair_sem_model_df['event_rating_num_mean'].mean()) ** 2).sum())
r2_joint = 1 - (ss_res / ss_tot) if ss_tot > 0 else np.nan
corr_joint = float(pair_sem_model_df['event_rating_num_mean'].corr(pair_sem_model_df['pred_event_rating_mean_joint'])) if len(pair_sem_model_df) > 1 else np.nan

print('Joint model (with intercept): event_rating_num_mean ≈ c + alpha*alignment_similarity + beta*semantic_similarity')
print(f'c_hat_joint: {c_hat_joint:.4f}')
print(f'alpha_hat_joint: {alpha_hat_joint:.4f}')
print(f'beta_hat_joint: {beta_hat_joint:.4f}')
print(f'RMSE: {rmse_joint:.4f}')
print(f'MAE: {mae_joint:.4f}')
print(f'R^2: {r2_joint:.4f}')
print(f'Pearson r: {corr_joint:.4f}')
pair_sem_model_df[['PairID', 'event_rating_num_mean', 'alignment_similarity', 'semantic_similarity', 'pred_event_rating_mean_joint']].head()


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Joint model: event_rating_num_mean ≈ alpha*alignment_similarity + beta*semantic_similarity
alpha_hat_joint: 0.4211
beta_hat_joint: 1.5693
RMSE: 1.3999
MAE: 1.2248
R^2: -6.8006


,PairID,event_rating_num_mean,alignment_similarity,semantic_similarity,semantic_similarity,pred_event_rating_mean_joint
0,P131,1.333333,0.00000,0.000000,0.000000,0.00000
1,P000,2.375000,0.00000,0.000000,0.000000,0.00000
2,P072,2.000000,0.00000,0.000000,0.000000,0.00000
3,P212,2.142857,0.00000,0.000000,0.000000,0.00000
4,P292,2.300000,0.96875,0.632487,0.632487,2.39309


## Interpretation: Joint Model (With Intercept)

- Fitted equation: `event_rating_num_mean ≈ 1.5672 + 0.1376 * alignment_similarity + 0.9832 * semantic_similarity`.
- `RMSE = 0.3883`, `MAE = 0.3208`, `R^2 = 0.3999`, `Pearson r = 0.6324`.
- Compared to alignment-only with intercept (`RMSE 0.4136`, `R^2 0.3192`), the joint model improves fit and explained variance.
- `beta_hat_joint` is larger than `alpha_hat_joint`, suggesting semantic similarity contributes more than alignment similarity in this linear specification.

Overall, adding semantic similarity provides a clear incremental gain over structure-only prediction.